In [1]:
import geopandas as gpd
import pandas as pd
import jupyter_black
import logging
import re
import os

In [2]:
jupyter_black.load()
logging.getLogger().setLevel(logging.INFO)

# Поиск населенный пункт - регион

In [25]:
import pandas as pd
from pyrosm import OSM, get_data

# Загружаем данные из OSM PBF файла
osm = OSM("C:/Users/Honor/Downloads/crimean-fed-district-latest.osm.pbf")
# Получаем все точки интереса (POIs) из OSM файла
pois = osm.get_pois()

# Извлекаем нужные данные (наименование региона и населенного пункта)
data = []
for idx, poi in pois.iterrows():
    if "addr:region" in poi and "addr:city" in poi:
        region = poi["addr:region"]
        city = poi["addr:city"]
        data.append([region, city])

# Создаем датафрейм с данными
df = pd.DataFrame(data, columns=["Region", "City"])

# Сохраняем датафрейм в файл (если нужно)
# df.to_csv("regions_and_cities.csv", index=False)

# Выводим первые несколько строк датафрейма для проверки
print(df.head())

Empty DataFrame
Columns: [Region, City]
Index: []


In [40]:
boundaries = osm.get_boundaries()

In [41]:
boundaries.columns

Index(['name', 'visible', 'admin_level', 'boundary', 'id', 'timestamp',
       'version', 'geometry', 'tags', 'osm_type', 'addr:country',
       'addr:postcode', 'ref', 'website', 'postal_code', 'border_type',
       'changeset'],
      dtype='object')

In [47]:
boundaries[boundaries["admin_level"] == "3"]

,name,visible,admin_level,boundary,id,timestamp,version,geometry,tags,osm_type,addr:country,addr:postcode,ref,website,postal_code,border_type,changeset
7,Южный федеральный округ,NaN,3,administrative,3649845,1722332321,518,"MULTIPOLYGON (((37.26877 44.5845, 37.2607 44.5...","{""name:ar"":""\u0645\u0646\u0637\u0642\u0629 \u0...",relation,RU,None,None,None,None,None,0.0


In [53]:
from pyrosm import OSM
import geopandas as gpd

filename = "donetsk_oblast-latest"
# Укажите путь к файлу PBF
pbf_path = "C:/Users/Honor/Downloads/crimean-fed-district-latest.osm.pbf"

# Загружаем данные OSM
osm = OSM(pbf_path)

# Извлекаем полигоны населенных пунктов (города, деревни и т.д.)
places = osm.get_data_by_custom_criteria(
    custom_filter={"region": True},
    filter_type="keep",
)

C:\Users\Honor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyrosm\pyrosm.py:767: UserWarning: Could not find any OSM data for given area.
  gdf = get_user_defined_data(


In [57]:
places

In [ ]:
# Фильтруем только полигоны
places_polygons = places[places.geometry.type == "Polygon"]

# Опционально: сохраняем результат в файл
places_polygons.to_file(f"{filename}.geojson", driver="GeoJSON")

In [ ]:
import osmnx as ox
import pandas as pd
import geopandas as gpd

# Загружаем данные из файла .pbf
gdf = ox.geometries_from_place("Russia", tags={"admin_level": "4"})

# Извлекаем наименование региона и населенного пункта
# Предполагается, что name для admin_level=4 является названием региона
# А для населенных пунктов - это города или села

# Фильтруем регионы (admin_level=4)
regions = gdf[gdf['admin_level'] == '4'][['name', 'geometry']]

# Загружаем все населенные пункты (admin_level=8)
places = ox.geometries_from_place("Russia", tags={"place": True})
places = places[['name', 'geometry']]

# Преобразуем данные в DataFrame
regions_df = pd.DataFrame(regions[['name']])
places_df = pd.DataFrame(places[['name']])

# Объединяем регионы и населенные пункты
result_df = pd.concat([regions_df, places_df], axis=1)

# Сохраняем результат в файл или выводим на экран
result_df.to_csv("regions_places.csv", index=False)
print(result_df.head())

In [3]:
from pyrosm import OSM
import geopandas as gpd

filename = "crimean-fed-district-latest"
# Укажите путь к файлу PBF
pbf_path = rf"C:/Users/Honor/Downloads/{filename}.osm.pbf"

# Загружаем данные OSM
osm = OSM(pbf_path)

# Извлекаем полигоны населенных пунктов (города, деревни и т.д.)
places = osm.get_data_by_custom_criteria(
    custom_filter={
        "place": ["city", "town", "village", "hamlet", "locality", "suburb"]
    },
    filter_type="keep",
)

# Фильтруем только полигоны
places_polygons = places[places.geometry.type == "Polygon"]

# Опционально: сохраняем результат в файл
places_polygons.to_file(f"{filename}.geojson", driver="GeoJSON")

INFO:pyogrio._io:Created 1,048 records


In [6]:
region = gpd.read_file(f"{filename}.geojson")

In [8]:
region.head(2)

,version,visible,timestamp,tags,id,changeset,lon,lat,place,osm_type,geometry
0,7,None,1720729731,"{ ""visible"": false, ""name"": ""Егорово"", ""name:e...",30711501,NaN,None,None,village,way,"POLYGON ((36.48503 45.38541, 36.48073 45.38464..."
1,16,None,1680157447,"{ ""visible"": false, ""name"": ""Станционное"", ""na...",30711507,NaN,None,None,village,way,"POLYGON ((36.03743 45.3534, 36.03265 45.35399,..."


In [ ]:
region[region.tags.str.contains("Крым")]

In [ ]:
    def way(self, w):
        tags = w.tags
        if 'boundary' in tags and tags['boundary'] == 'administrative':
            if 'name' in tags and 'admin_level' in tags:
                name = tags['name']
                admin_level = tags['admin_level']

                if admin_level == '4':  # Уровень региона (обычно admin_level = 4 для регионов)
                    region_name = name

                    # Получаем геометрию и сохраняем её как WKT
                    geom_wkt = w.geom.wkt
                    self.data.append({
                        'region': region_name,
                        'geometry': geom_wkt
                    })

        if 'place' in tags and tags['place'] in ['city', 'town', 'village']:
            if 'name' in tags:
                place_name = tags['name']

                # Получаем геометрию и сохраняем её как WKT
                geom_wkt = w.geom.wkt
                self.data.append({
                    'city': place_name,
                    'geometry': geom_wkt
                })

    def save_to_file(self, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            for item in self.data:
                if 'region' in item:
                    f.write(f"Регион: {item['region']}, Геометрия: {item['geometry']}\n")
                elif 'city' in item:
                    f.write(f"Населенный пункт: {item['city']}, Геометрия: {item['geometry']}\n")

# Инициализация обработчика
handler = OSMHandler()

# Путь к файлу PBF
pbf_file = "C:/Users/Honor/Downloads/crimean-fed-district-latest.osm.pbf"

# Обработка файла
handler.apply_file(pbf_file)
print(handler)
# Сохранение результата в файл
# handler.save_to_file('output.txt')

print("Обработка завершена. Результат сохранен в 'output.txt'")

In [ ]:
handler